In [1]:
import json, re
from pathlib import Path
# from statistics import median
from collections import Counter
import numpy as np

ROOT_DIR = Path(r"D:\ex\최종프로젝트\AI_Dev_3rd_Project\CHOI\feellog-project_03\reports\analysis")

In [2]:
def scan(date_range: dict, root_dir: Path):
    """
    지정된 기간(date_range) 안의 analysis_result_*.json 파일들을 스캔
    return: dict[date] -> list[Path]
    """
    start = date_range.get("start")
    end   = date_range.get("end")
    files_by_day = {}

    for p in Path(root_dir).glob("analysis_result_*.json"):
        m = re.search(r"(\d{8})", p.name)
        if not m:
            continue
        d = m.group(1)
        day = f"{d[:4]}-{d[4:6]}-{d[6:]}"   # YYYY-MM-DD
        if start <= day <= end:
            files_by_day.setdefault(day, []).append(p)
    return files_by_day



In [3]:
EMOTIONS = ["기쁨","당황","분노","불안","상처","슬픔","중립"]

def load_and_normalize(path: Path):
    """
    파일에서 감정 결과를 읽어 7감정 분포로 확장/정규화
    return: dict {score:int, top:str, dist:dict}
    """
    with open(path, encoding="utf-8") as f:
        raw = json.load(f)
    score = raw.get("sentiment_score")
    top = raw.get("dominant_overall_emotion")
    
    dist = {e:0.0 for e in EMOTIONS}
    for item in raw.get("emotion_distribution", []):
        emo = item["emotion"]
        val = float(item["percentage"].replace("%",""))/100
        if emo in dist:
            dist[emo] += val
    # 정규화
    s = sum(dist.values())
    if s>0:
        dist = {k:v/s for k,v in dist.items()}
    return {"score":score, "top":top, "dist":dist}


In [4]:
def aggregate_daily(paths:list[Path]):
    """
    하루 여러 파일 → DailyCardJSON
    """
    recs = [load_and_normalize(p) for p in paths]
    
    scores = [r["score"] for r in recs if r["score"] is not None]
    score = round(np.mean(scores)) if scores else None
    
    tops = [r["top"] for r in recs if r["top"]]
    top = Counter(tops).most_common(1)[0][0] if tops else None
    
    dist_sum = {e:0 for e in EMOTIONS}
    for r in recs:
        for e,v in r["dist"].items():
            dist_sum[e]+=v
    s = sum(dist_sum.values())
    if s>0:
        dist_sum = {k:v/s for k,v in dist_sum.items()}
    
    return {"score":score,"top":top,"dist":dist_sum}


In [5]:
def aggregate_period(cards:dict[str,dict]):
    """
    여러 일간 카드 → 기간 리포트 JSON
    """
    if not cards: return {}
    
    scores = [c["score"] for c in cards.values() if c["score"]]
    avg_score = round(np.mean(scores)) if scores else None
    
    tops = [c["top"] for c in cards.values() if c["top"]]
    top_counts = Counter(tops)
    doms = [{"emotion":k,"percentage":v/len(tops)} for k,v in top_counts.items()]
    
    trend = [{"date":d,"score":c["score"],"top":c["top"]} for d,c in sorted(cards.items())]
    
    # 급변 탐지
    notable=[]
    for i in range(1,len(trend)):
        delta = trend[i]["score"]-trend[i-1]["score"]
        if abs(delta)>=10:
            notable.append({"date":trend[i]["date"],"event":"점수 급변","delta":delta})
        if trend[i]["top"]!=trend[i-1]["top"]:
            notable.append({"date":trend[i]["date"],"event":f"주감정 전환: {trend[i-1]['top']}→{trend[i]['top']}"})
    
    return {"average_sentiment_score":avg_score,
            "dominant_emotions":doms,
            "trend":{"by_day":trend,"notable_changes":notable}}


In [6]:
# 1. intent 모듈 결과 예시
date_range = {
    "mode": "weekly",
    "date": None,
    "start": "2025-08-18",
    "end": "2025-08-26",
    "source": "저번주인가 일이 많아서 우울했던거 같은데 어땠더라?"
}

# 2. 스캔
files = scan(date_range, ROOT_DIR)

# 3. 일간 집계
daily_cards = {d:aggregate_daily(ps) for d,ps in files.items()}

# 4. 기간 집계
period = aggregate_period(daily_cards)


In [7]:
daily_cards

{'2025-08-18': {'score': 20,
  'top': '상처',
  'dist': {'기쁨': 0.0,
   '당황': 0.26522981755539893,
   '분노': 0.30280322605904003,
   '불안': 0.0,
   '상처': 0.43196695638556104,
   '슬픔': 0.0,
   '중립': 0.0}},
 '2025-08-21': {'score': 35,
  'top': '중립',
  'dist': {'기쁨': 0.0,
   '당황': 0.1388888888888889,
   '분노': 0.16666666666666666,
   '불안': 0.0,
   '상처': 0.19444444444444442,
   '슬픔': 0.0,
   '중립': 0.5}},
 '2025-08-23': {'score': 30,
  'top': '상처',
  'dist': {'기쁨': 0.0,
   '당황': 0.0,
   '분노': 0.3125,
   '불안': 0.0,
   '상처': 0.43749999999999994,
   '슬픔': 0.25,
   '중립': 0.0}},
 '2025-08-24': {'score': 60,
  'top': '기쁨',
  'dist': {'기쁨': 0.45,
   '당황': 0.25,
   '분노': 0.0,
   '불안': 0.3,
   '상처': 0.0,
   '슬픔': 0.0,
   '중립': 0.0}}}

In [8]:
period

{'average_sentiment_score': 36,
 'dominant_emotions': [{'emotion': '상처', 'percentage': 0.5},
  {'emotion': '중립', 'percentage': 0.25},
  {'emotion': '기쁨', 'percentage': 0.25}],
 'trend': {'by_day': [{'date': '2025-08-18', 'score': 20, 'top': '상처'},
   {'date': '2025-08-21', 'score': 35, 'top': '중립'},
   {'date': '2025-08-23', 'score': 30, 'top': '상처'},
   {'date': '2025-08-24', 'score': 60, 'top': '기쁨'}],
  'notable_changes': [{'date': '2025-08-21', 'event': '점수 급변', 'delta': 15},
   {'date': '2025-08-21', 'event': '주감정 전환: 상처→중립'},
   {'date': '2025-08-23', 'event': '주감정 전환: 중립→상처'},
   {'date': '2025-08-24', 'event': '점수 급변', 'delta': 30},
   {'date': '2025-08-24', 'event': '주감정 전환: 상처→기쁨'}]}}